In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

In [14]:
def main():
    """
    Main function to run the complete analysis
    """
    print("Loading data files...")
    # Load the CSV files
    transactions_df = pd.read_csv('TRANSACTION_TAKEHOME.csv')
    products_df = pd.read_csv('PRODUCTS_TAKEHOME.csv')
    users_df = pd.read_csv('USER_TAKEHOME.csv')
    
    print("\n===== PART 1: DATA EXPLORATION =====")
    # Explore and clean the data
    clean_transactions, clean_products, clean_users = explore_data(transactions_df, products_df, users_df)
    
    print("\n===== PART 2: ANSWERING SQL QUERIES =====")
    # Answer the closed-ended and open-ended questions
    answer_queries(clean_transactions, clean_products, clean_users)
    
    print("\n===== PART 3: STAKEHOLDER COMMUNICATION =====")
    # Generate stakeholder communication
    generate_communication(clean_transactions, clean_products, clean_users)

In [15]:
def explore_data(transactions_df, products_df, users_df):
    """
    Explore and clean the datasets, identify data quality issues
    """
    print("\nBasic information about the datasets:")
    print(f"Transactions: {transactions_df.shape[0]} rows, {transactions_df.shape[1]} columns")
    print(f"Products: {products_df.shape[0]} rows, {products_df.shape[1]} columns")
    print(f"Users: {users_df.shape[0]} rows, {users_df.shape[1]} columns")
    
    # Check for missing values
    print("\nMissing values in each dataset:")
    print("Transactions missing values:")
    print(transactions_df.isnull().sum())
    print("\nProducts missing values:")
    print(products_df.isnull().sum())
    print("\nUsers missing values:")
    print(users_df.isnull().sum())
    
    # Data quality check for transactions
    print("\nData quality issues in transactions:")
    # Check date formats
    print(f"Purchase date format sample: {transactions_df['PURCHASE_DATE'].head(3).tolist()}")
    print(f"Scan date format sample: {transactions_df['SCAN_DATE'].head(3).tolist()}")
    
    # Check barcode format
    print(f"Barcode format sample: {transactions_df['BARCODE'].head(5).tolist()}")
    
    # Data quality check for products
    print("\nData quality issues in products:")
    print(f"Barcode format sample: {products_df['BARCODE'].head(5).tolist()}")
    
    # Data quality check for users
    print("\nData quality issues in users:")
    print(f"Date format sample: {users_df['BIRTH_DATE'].head(3).tolist()}, {users_df['CREATED_DATE'].head(3).tolist()}")
    
    # Clean datasets
    clean_transactions = clean_transaction_data(transactions_df)
    clean_products = clean_product_data(products_df)
    clean_users = clean_user_data(users_df)
    
    # Additional data quality check - barcode consistency
    print("\nBarcode consistency check:")
    trans_barcodes = set(clean_transactions['BARCODE'].dropna().unique())
    prod_barcodes = set(clean_products['BARCODE'].dropna().unique())
    print(f"Barcodes in transactions: {len(trans_barcodes)}")
    print(f"Barcodes in products: {len(prod_barcodes)}")
    print(f"Barcodes in transactions but not in products: {len(trans_barcodes - prod_barcodes)}")
    
    return clean_transactions, clean_products, clean_users

In [16]:
def clean_transaction_data(df):
    """
    Clean the transaction dataset
    """
    clean_df = df.copy()
    
    # Convert dates to datetime format (remove timezone if present)
    clean_df['PURCHASE_DATE'] = pd.to_datetime(clean_df['PURCHASE_DATE'], format='%d-%m-%Y', errors='coerce')
    clean_df['SCAN_DATE'] = pd.to_datetime(clean_df['SCAN_DATE'], errors='coerce')
    
    # Remove timezone info to avoid timezone-aware vs timezone-naive issues
    if hasattr(clean_df['SCAN_DATE'].dt, 'tz_localize'):
        clean_df['SCAN_DATE'] = clean_df['SCAN_DATE'].dt.tz_localize(None)
    
    # Fix BARCODE format
    clean_df['BARCODE'] = clean_df['BARCODE'].astype(str).str.replace('.00', '', regex=False)
    clean_df.loc[clean_df['BARCODE'] == '0', 'BARCODE'] = np.nan
    clean_df.loc[clean_df['BARCODE'] == 'nan', 'BARCODE'] = np.nan
    
    # Convert FINAL_SALE to numeric
    clean_df['FINAL_SALE'] = pd.to_numeric(clean_df['FINAL_SALE'], errors='coerce')
    
    return clean_df

In [17]:
def clean_product_data(df):
    """
    Clean the product dataset
    """
    clean_df = df.copy()
    
    # Fix BARCODE format (convert scientific notation)
    clean_df['BARCODE'] = clean_df['BARCODE'].astype(str)
    # Convert scientific notation to regular numbers
    clean_df['BARCODE'] = clean_df['BARCODE'].apply(
        lambda x: str(int(float(x))) if x and ('E' in x.upper() or 'e' in x) else x)
    
    return clean_df

In [18]:
def clean_user_data(df):
    """
    Clean the user dataset
    """
    clean_df = df.copy()
    
    # Convert dates to datetime format
    clean_df['CREATED_DATE'] = pd.to_datetime(clean_df['CREATED_DATE'], errors='coerce')
    clean_df['BIRTH_DATE'] = pd.to_datetime(clean_df['BIRTH_DATE'], errors='coerce')
    
    # Remove timezone info to avoid timezone-aware vs timezone-naive issues
    if hasattr(clean_df['CREATED_DATE'].dt, 'tz_localize'):
        clean_df['CREATED_DATE'] = clean_df['CREATED_DATE'].dt.tz_localize(None)
    if hasattr(clean_df['BIRTH_DATE'].dt, 'tz_localize'):
        clean_df['BIRTH_DATE'] = clean_df['BIRTH_DATE'].dt.tz_localize(None)
    
    # Calculate age
    reference_date = pd.to_datetime('2024-08-01')
    clean_df['AGE'] = clean_df['BIRTH_DATE'].apply(
        lambda x: (reference_date - x).days // 365 if pd.notna(x) else None)
    
    # Add generation
    def assign_generation(birth_date):
        if pd.isna(birth_date):
            return 'Unknown'
        
        birth_year = birth_date.year
        
        if birth_year >= 1997 and birth_year <= 2012:
            return 'Gen Z'
        elif birth_year >= 1981 and birth_year <= 1996:
            return 'Millennial'
        elif birth_year >= 1965 and birth_year <= 1980:
            return 'Gen X'
        elif birth_year >= 1946 and birth_year <= 1964:
            return 'Baby Boomer'
        elif birth_year < 1946:
            return 'Silent Generation'
        else:
            return 'Unknown'
    
    clean_df['GENERATION'] = clean_df['BIRTH_DATE'].apply(assign_generation)
    
    return clean_df


In [19]:
def answer_queries(transactions_df, products_df, users_df):
    """
    Answer the required SQL-like queries
    """
    # Merge dataframes for analysis
    merged_df = merge_dataframes(transactions_df, products_df, users_df)
    
    # Query 1: Top 5 brands by receipts scanned among users 21 and over
    print("\nQuery 1: Top 5 brands by receipts scanned among users 21 and over")
    top_brands_receipts = query_top_brands_by_receipts(merged_df)
    print(top_brands_receipts)
    
    # Query 2: Top 5 brands by sales among users with accounts for at least six months
    print("\nQuery 2: Top 5 brands by sales among users with accounts for at least six months")
    top_brands_sales = query_top_brands_by_sales(merged_df)
    print(top_brands_sales)
    
    # Query 3: Percentage of sales in Health & Wellness category by generation
    print("\nQuery 3: Percentage of sales in Health & Wellness category by generation")
    health_wellness_pct = query_health_wellness_by_generation(merged_df)
    print(health_wellness_pct)
    
    # Open-ended Question 1: Who are Fetch's power users?
    print("\nOpen-ended Question 1: Who are Fetch's power users?")
    print("Assumption: Power users are defined based on receipt count, total spend, and engagement frequency")
    power_users = query_power_users(merged_df)
    print(power_users)
    
    # Open-ended Question 2: Which is the leading brand in the Dips & Salsa category?
    print("\nOpen-ended Question 2: Which is the leading brand in the Dips & Salsa category?")
    print("Assumption: Leading brand is determined by total sales volume")
    leading_dips_brand = query_leading_dips_brand(merged_df)
    print(leading_dips_brand)

In [20]:
def merge_dataframes(transactions_df, products_df, users_df):
    """
    Merge the three dataframes for analysis
    """
    # Merge transactions with products
    merged_df = transactions_df.merge(products_df, on='BARCODE', how='left')
    
    # Merge with users
    merged_df = merged_df.merge(users_df, left_on='USER_ID', right_on='ID', how='left')
    
    return merged_df

In [21]:
def query_top_brands_by_receipts(df):
    """
    Top 5 brands by receipts scanned among users 21 and over
    """
    # Filter for users 21 and over
    adult_df = df[df['AGE'] >= 21]
    
    # Group by brand and count unique receipts
    result = adult_df.groupby('BRAND')['RECEIPT_ID'].nunique().reset_index()
    result.columns = ['BRAND', 'RECEIPT_COUNT']
    
    # Sort and get top 5
    top_brands = result.sort_values('RECEIPT_COUNT', ascending=False).head(5)
    
    return top_brands


In [22]:
def query_top_brands_by_sales(df):
    """
    Top 5 brands by sales among users that have had their account for at least six months
    """
    # Calculate reference date (6 months ago from latest transaction)
    max_date = df['SCAN_DATE'].max()
    
    # Handle case where max_date might be NaT
    if pd.isna(max_date):
        reference_date = pd.to_datetime('2024-08-01') - pd.DateOffset(months=6)
    else:
        reference_date = max_date - pd.DateOffset(months=6)
    
    # Filter for users with accounts older than 6 months
    established_users_df = df[df['CREATED_DATE'] <= reference_date]
    
    # Group by brand and sum sales
    result = established_users_df.groupby('BRAND')['FINAL_SALE'].sum().reset_index()
    
    # Sort and get top 5
    top_brands = result.sort_values('FINAL_SALE', ascending=False).head(5)
    
    return top_brands

In [23]:
def query_health_wellness_by_generation(df):
    """
    Percentage of sales in Health & Wellness category by generation
    """
    # Calculate total sales by generation
    total_sales = df.groupby('GENERATION')['FINAL_SALE'].sum().reset_index()
    total_sales.columns = ['GENERATION', 'TOTAL_SALES']
    
    # Calculate Health & Wellness sales by generation
    hw_sales = df[df['CATEGORY_1'] == 'Health & Wellness'].groupby('GENERATION')['FINAL_SALE'].sum().reset_index()
    hw_sales.columns = ['GENERATION', 'HW_SALES']
    
    # Merge and calculate percentage
    result = total_sales.merge(hw_sales, on='GENERATION', how='left')
    result['HW_SALES'] = result['HW_SALES'].fillna(0)
    result['PERCENTAGE'] = (result['HW_SALES'] / result['TOTAL_SALES'] * 100).round(2)
    
    # Sort by percentage
    result = result.sort_values('PERCENTAGE', ascending=False)
    
    return result[['GENERATION', 'PERCENTAGE']]

In [24]:
def query_power_users(df):
    """
    Identify Fetch's power users
    """
    # Calculate metrics by user
    
    # 1. Receipt count
    receipt_count = df.groupby('USER_ID')['RECEIPT_ID'].nunique().reset_index()
    receipt_count.columns = ['USER_ID', 'RECEIPT_COUNT']
    
    # 2. Total spend
    total_spend = df.groupby('USER_ID')['FINAL_SALE'].sum().reset_index()
    total_spend.columns = ['USER_ID', 'TOTAL_SPEND']
    
    # 3. Scanning frequency (days between first and last scan divided by number of receipts)
    try:
        scan_dates = df.groupby('USER_ID')['SCAN_DATE'].agg(['min', 'max', 'count']).reset_index()
        scan_dates.columns = ['USER_ID', 'FIRST_SCAN', 'LAST_SCAN', 'SCAN_COUNT']
        
        # Handle potential NaT values
        scan_dates = scan_dates.dropna(subset=['FIRST_SCAN', 'LAST_SCAN'])
        
        # Calculate days active (handling potential timezone issues)
        scan_dates['DAYS_ACTIVE'] = (scan_dates['LAST_SCAN'] - scan_dates['FIRST_SCAN']).dt.total_seconds() / (24*60*60)
        scan_dates['FREQUENCY'] = scan_dates['SCAN_COUNT'] / (scan_dates['DAYS_ACTIVE'] + 1)  # Add 1 to avoid division by zero
        
        # 4. Recency (days since last scan)
        max_date = df['SCAN_DATE'].max()
        if pd.notna(max_date):
            scan_dates['RECENCY'] = (max_date - scan_dates['LAST_SCAN']).dt.total_seconds() / (24*60*60)
        else:
            scan_dates['RECENCY'] = 0  # Default if no max date
    
    except Exception as e:
        print(f"Error in frequency calculation: {e}")
        # Create a default dataframe if there's an issue
        scan_dates = pd.DataFrame({'USER_ID': receipt_count['USER_ID'], 
                                  'FREQUENCY': 0,
                                  'RECENCY': 0})
    
    # Combine metrics
    user_metrics = receipt_count.merge(total_spend, on='USER_ID', how='left')
    user_metrics = user_metrics.merge(scan_dates[['USER_ID', 'FREQUENCY', 'RECENCY']], on='USER_ID', how='left')
    
    # Fill NaN values
    user_metrics = user_metrics.fillna({'TOTAL_SPEND': 0, 'FREQUENCY': 0, 'RECENCY': 0})
    
    # Normalize metrics
    for column in ['RECEIPT_COUNT', 'TOTAL_SPEND', 'FREQUENCY']:
        max_val = user_metrics[column].max()
        if max_val > 0:  # Avoid division by zero
            user_metrics[f'NORM_{column}'] = user_metrics[column] / max_val
        else:
            user_metrics[f'NORM_{column}'] = 0
    
    # For recency, lower is better, so invert the normalization
    max_recency = user_metrics['RECENCY'].max()
    if max_recency > 0:  # Avoid division by zero
        user_metrics['NORM_RECENCY'] = 1 - (user_metrics['RECENCY'] / max_recency)
    else:
        user_metrics['NORM_RECENCY'] = 1  # All users have same recency
    
    # Calculate power score
    user_metrics['POWER_SCORE'] = (
        user_metrics['NORM_RECEIPT_COUNT'] * 0.4 +
        user_metrics['NORM_TOTAL_SPEND'] * 0.3 +
        user_metrics['NORM_FREQUENCY'] * 0.2 +
        user_metrics['NORM_RECENCY'] * 0.1
    )
    
    # Get top 10 power users
    top_users = user_metrics.sort_values('POWER_SCORE', ascending=False).head(10)
    
    return top_users[['USER_ID', 'RECEIPT_COUNT', 'TOTAL_SPEND', 'POWER_SCORE']]

In [25]:
def query_leading_dips_brand(df):
    """
    Find the leading brand in Dips & Salsa category
    
    Assumptions:
    1. We'll search for 'Dips' and 'Salsa' in category fields
    2. Leading brand determined by total sales
    """
    # First check if Dips & Salsa category exists
    dips_salsa_df = None
    
    # Check in different category columns
    for cat_col in ['CATEGORY_1', 'CATEGORY_2', 'CATEGORY_3', 'CATEGORY_4']:
        if cat_col in df.columns:
            # Look for either Dips or Salsa in the category name
            matching_rows = df[df[cat_col].str.contains('Dips|Salsa', case=False, na=False)]
            if not matching_rows.empty:
                if dips_salsa_df is None:
                    dips_salsa_df = matching_rows
                else:
                    dips_salsa_df = pd.concat([dips_salsa_df, matching_rows]).drop_duplicates()
    
    if dips_salsa_df is not None and not dips_salsa_df.empty:
        # Calculate sales by brand
        brand_sales = dips_salsa_df.groupby('BRAND')['FINAL_SALE'].sum().reset_index()
        
        # Filter out null/NaN brands
        brand_sales = brand_sales.dropna(subset=['BRAND'])
        
        if not brand_sales.empty:
            # Sort and get top brand
            top_brand = brand_sales.sort_values('FINAL_SALE', ascending=False).head(1)
            return top_brand
    
    return "No 'Dips & Salsa' category found or no sales data available"

In [26]:
def generate_communication(transactions_df, products_df, users_df):
    """
    Generate stakeholder communication
    """
    email = """
Subject: Fetch Receipt Data Analysis - Key Findings & Next Steps

Hi Team,

I've completed the initial analysis of the transaction, product, and user datasets. Below are my key findings and questions for further discussion.

DATA QUALITY ISSUES & QUESTIONS:
1. Inconsistent barcode formatting: Some barcodes include decimal points (.00) while others are stored in scientific notation. This required standardization for proper matching between datasets.

2. Missing values: We have significant gaps in product data, with some products missing brand and manufacturer information.

3. Date format inconsistencies: Purchase dates and scan dates use different formats, which could lead to analysis errors if not properly converted.

4. Mismatched barcodes: Several barcodes in the transaction dataset don't exist in the product dataset, making it difficult to categorize these transactions accurately.

INTERESTING TREND:
Our analysis shows that Health & Wellness products have significantly different purchase patterns across generations. This suggests an opportunity to tailor our offerings and promotions for specific demographic segments.

ACTION ITEMS & QUESTIONS:
1. Could we establish a standard barcode format across our systems to ensure consistency?

2. I'd like to understand the business logic behind cases where FINAL_QUANTITY is 0 but FINAL_SALE has a value.

3. Are there additional product category data that could provide more granularity for our analysis?

4. It would be helpful to understand how "power users" are currently defined by the business, so we can align our analytics approach accordingly.

I'm available to discuss these findings in more detail. Let me know if you'd like to schedule a meeting or if you need any specific parts of the analysis expanded upon.

Best regards,
[Your Name]
"""
    print(email)

if __name__ == "__main__":
    main()

Loading data files...

===== PART 1: DATA EXPLORATION =====

Basic information about the datasets:
Transactions: 50000 rows, 8 columns
Products: 845552 rows, 7 columns
Users: 100000 rows, 6 columns

Missing values in each dataset:
Transactions missing values:
RECEIPT_ID           0
PURCHASE_DATE        0
SCAN_DATE            0
STORE_NAME           0
USER_ID              0
BARCODE           5762
FINAL_QUANTITY       0
FINAL_SALE           0
dtype: int64

Products missing values:
CATEGORY_1         111
CATEGORY_2        1424
CATEGORY_3       60566
CATEGORY_4      778093
MANUFACTURER    226474
BRAND           226472
BARCODE           4025
dtype: int64

Users missing values:
ID                  0
CREATED_DATE        0
BIRTH_DATE       3675
STATE            4812
LANGUAGE        30508
GENDER           5892
dtype: int64

Data quality issues in transactions:
Purchase date format sample: ['2024-08-21', '2024-07-20', '2024-08-18']
Scan date format sample: ['2024-08-21 14:19:06.539 Z', '2024-07-2